In [2]:
#import required libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup


In [4]:
#request html content, both for regular season and playoffs
url_base_reg = 'https://www.basketball-reference.com/leagues/NBA_{}.html'
url_base_play = 'https://www.basketball-reference.com/playoffs/NBA_{}.html'
url_base_payroll = 'https://www.spotrac.com/nba/tax/{}'

years = [2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013]

teams = ["Boston Celtics", "Brooklyn Nets", "New York Knicks", "Philadelphia 76ers", "Toronto Raptors", "Chicago Bulls",
"Cleveland Cavaliers", "Detroit Pistons", "Indiana Pacers", "Milwaukee Bucks", "Atlanta Hawks", "Charlotte Hornets",
"Miami Heat", "Orlando Magic", "Washington Wizards", "Denver Nuggets", "Minnesota Timberwolves", "Oklahoma City Thunder",
"Portland Trail Blazers", "Utah Jazz", "Golden State Warriors", "Los Angeles Clippers", "Los Angeles Lakers", "Phoenix Suns",
"Sacramento Kings", "Dallas Mavericks", "Houston Rockets", "Memphis Grizzlies", "New Orleans Pelicans", "San Antonio Spurs", "Charlotte Bobcats", "New Orleans Hornets"]

df_main = pd.DataFrame()

df_payroll = pd.DataFrame(columns=["Team", "Active"])

cpi = pd.read_csv('data/cpi-u_2013-22.csv')
bri = pd.read_csv('data/total_league_revenues.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/cpi-u_2013-22.csv'

In [ ]:
for i in years:

    url_reg = url_base_reg.format(i)

    per_game = requests.get(url_reg)

    #create BeautifulSoup object
    soup = BeautifulSoup(per_game.content, 'html.parser')

    #locate correct table
    table = soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=="per_game-team") 
    rows = table.findAll(lambda tag: tag.name=='tr')

    #create DataFrame
    df = pd.read_html(str(table))[0]

    #clean DataFrame
    df = df.drop(index=30)
    df['Team'] = df['Team'].str.replace('*', '')
    df.insert(2, "Year", i, True)
    df = df.drop(columns=["G","Rk"])
    df = df.reset_index(drop=True)

    #add advanced stats to DataFrame
    table = soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=="advanced-team") 
    rows = table.findAll(lambda tag: tag.name=='tr')
    df_adv = pd.read_html(str(table))[0]
    df_adv.columns = df_adv.columns.droplevel()
    df_adv = df_adv.drop(index=30)
    df_adv['Team'] = df_adv['Team'].str.replace('*', '')
    df_adv = df_adv.drop(columns=["Rk", "L", "PW", "PL", "Unnamed: 17_level_1", "Unnamed: 22_level_1", "Unnamed: 27_level_1", "Arena"])
    df_adv = df_adv.reset_index(drop=True)

    #add playoff wins to DataFrame
    url_play = url_base_play.format(i)
    playoff_wins = requests.get(url_play)
    soup = BeautifulSoup(playoff_wins.content, 'html.parser')
    table = soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=="advanced-team")
    df_play = pd.read_html(str(table))[0]
    df_play.columns = df_play.columns.droplevel()
    df_play = df_play.drop(index=16)
    df_play = df_play.rename(columns={"Tm": "Team", "W" : "Playoff_W"})
    df_play = df_play[["Team", "Playoff_W"]]

    #add payroll data
    url_payroll = url_base_payroll.format(i)

    payrolls = requests.get(url_payroll)
    
    soup = BeautifulSoup(payrolls.content, 'html.parser')

    table = soup.find("table", class_="datatable rtable captracker")

    for row in table.tbody.find_all('tr'):
        columns = row.find_all('td')

        if(columns != []):
            team = columns[1].text[:-3]
            active = int(columns[3].text.strip("$").replace(',',''))
            df_payroll = df_payroll.append({"Team": team, "Active": active}, ignore_index=True)
    
    # calculate payroll over average
    df_payroll['Adjusted Active'] = df_payroll['Active'].apply(lambda x: x/(cpi.loc[cpi['Year'] is i]['Annual'] + bri.loc[bri['Year'] is i]['Revenue']))

    # merge all datasets
    df = pd.merge(df, df_adv, on='Team')
    df = pd.merge(df, df_play, on='Team')
    df = pd.merge(df, df_payroll, on='Team')

    df_main = pd.concat([df_main, df])


In [ ]:
df_main

In [ ]:
#write data to csv
df_main.to_csv('playoff_per_game_2013-22.csv')